In [5]:
!git clone https://github.com/muellerzr/ML_Useful_Functions.git
from ML_Useful_Functions.Functions import *

Cloning into 'ML_Useful_Functions'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 33 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [0]:
class SaveModelCallback(TrackerCallback):
    "A `TrackerCallback` that saves the model when monitored quantity is best."
    def __init__(self, learn:Learner, monitor:str='accuracy', mode:str='auto', every:str='improvement', name:str='bestmodel',):
        super().__init__(learn, monitor=monitor, mode=mode)
        self.every,self.name = every,name
        if self.every not in ['improvement', 'epoch']:
            warn(f'SaveModel every {self.every} is invalid, falling back to "improvement".')
            self.every = 'improvement'

                 
    def jump_to_epoch(self, epoch:int)->None:
        try: 
            self.learn.load(f'{self.name}_{epoch-1}', purge=False)
            print(f"Loaded {self.name}_{epoch-1}")
        except: print(f'Model {self.name}_{epoch-1} not found.')

    def on_epoch_end(self, epoch:int, **kwargs:Any)->None:
        "Compare the value monitored to its best score and maybe save the model."
        if self.every=="epoch": self.learn.save(f'{self.name}_{epoch}')
        else: #every="improvement"
            current = self.get_monitor_value()
            if current is not None and self.operator(current, self.best):
                #print(f'Better model found at epoch {epoch} with {self.monitor} value: {current}.')
                self.best = current
                self.learn.save(f'{self.name}')

    def on_train_end(self, **kwargs):
        "Load the best model."
        if self.every=="improvement" and (self.learn.path/f'{self.learn.model_dir}/{self.name}.pth').is_file():
            self.learn.load(f'{self.name}', purge=False)
            current = self.get_monitor_value()
            print(float(current))
            

In [0]:
from fastai.tabular import *
from fastai.callbacks.tracker import *
from fastai.utils.mod_display import *

In [0]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [0]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [0]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           
                           .databunch())

# Here I want to find the best Alpha value

i is your initial value you want to modify. Within the learn call, modify what you want to change.

In [38]:
i = 1
for x in range(10):
  learn = tabular_learner(data, layers=calcHiddenLayer(data, i, 2), metrics=accuracy, callback_fns=SaveModelCallback)
  if x == 0:
    print('Alpha:', i)
  else:
    print('\nAlpha:', i)
  with progress_disabled_ctx(learn) as learn:
    learn.fit_one_cycle(5)
  i += 1

Alpha: 1
0.8349999785423279

Alpha: 2
0.8450000286102295

Alpha: 3
0.8349999785423279

Alpha: 4
0.8299999833106995

Alpha: 5
0.8399999737739563

Alpha: 6
0.8299999833106995

Alpha: 7
0.8299999833106995

Alpha: 8
0.8299999833106995

Alpha: 9
0.8450000286102295

Alpha: 10
0.8349999785423279
